### LIBRARIES

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import string
import random

### DATA LOADING AND VISUALISATION

In [9]:
import pandas as pd
from collections import Counter
# Load the CSV file
df = pd.read_csv("/kaggle/input/info-sec-a2/Balanced_dataset.csv")

# Display basic information
print(df.head())  # Show first 5 rows
print(df.info())  # Get data types and non-null values


                                                 url    type scheme  \
0    https://ethnicelebs.com/eddie-anderson-comedian  benign  https   
1  https://closinglogos.com/page/Sony+Pictures+Ho...  benign  https   
2  https://terezowens.com/golfer-greg-norman-tryi...  benign  https   
3                        https://americannortel.com/  benign  https   
4                               https://welding.org/  benign  https   

               domain subdomain top_level_domain  \
0     ethnicelebs.com      none              com   
1    closinglogos.com      none              com   
2      terezowens.com      none              com   
3  americannortel.com      none              com   
4         welding.org      none              org   

                                                path  path_length  \
0                           /eddie-anderson-comedian           24   
1  /page/Sony+Pictures+Home+Entertainment+Warning...           53   
2  /golfer-greg-norman-trying-to-unload-55-millio... 

In [10]:
df

,url,type,scheme,domain,subdomain,top_level_domain,path,path_length,num_path_segments,query_params,num_query_params,has_https,file_extension,has_fragment,has_special_chars_in_path,has_port,port_number,is_ip_address
0,https://ethnicelebs.com/eddie-anderson-comedian,benign,https,ethnicelebs.com,none,com,/eddie-anderson-comedian,24,1,0,0,1,none,0,0,0,0,0
1,https://closinglogos.com/page/Sony+Pictures+Ho...,benign,https,closinglogos.com,none,com,/page/Sony+Pictures+Home+Entertainment+Warning...,53,2,0,0,1,none,0,0,0,0,0
2,https://terezowens.com/golfer-greg-norman-tryi...,benign,https,terezowens.com,none,com,/golfer-greg-norman-trying-to-unload-55-millio...,54,1,0,0,1,none,0,0,0,0,0
3,https://americannortel.com/,benign,https,americannortel.com,none,com,/,1,0,0,0,1,none,0,0,0,0,0
4,https://welding.org/,benign,https,welding.org,none,org,/,1,0,0,0,1,none,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59600,http://archives.li.man.ac.uk/ead/search/?opera...,spam,http,archives.li.man.ac.uk,archives,uk,/ead/search/,12,2,6,6,0,none,0,0,0,0,0
59601,http://amber.ch.ic.ac.uk/archive/all/10253.html,spam,http,amber.ch.ic.ac.uk,amber,uk,/archive/all/10253.html,23,3,0,0,0,html,0,0,0,0,0
59602,http://archive.thisischeshire.co.uk/2000/1/28/...,spam,http,archive.thisischeshire.co.uk,archive,uk,/2000/1/28/225238.html,22,4,0,0,0,html,0,0,0,0,0
59603,http://ads.guardian.co.uk/click.ng/Params.rich...,spam,http,ads.guardian.co.uk,ads,uk,/click.ng/Params.richmedia=yes&location=middle...,128,2,0,0,0,richmedia=yes&location=middle&spacedesc=06&sit...,0,1,0,0,0


In [11]:
df.columns

Index(['url', 'type', 'scheme', 'domain', 'subdomain', 'top_level_domain',
       'path', 'path_length', 'num_path_segments', 'query_params',
       'num_query_params', 'has_https', 'file_extension', 'has_fragment',
       'has_special_chars_in_path', 'has_port', 'port_number',
       'is_ip_address'],
      dtype='object')

In [12]:
df['type'].value_counts()

type
benign        11921
defacement    11921
phishing      11921
malware       11921
spam          11921
Name: count, dtype: int64

In [13]:
df[df.columns][:5]

,url,type,scheme,domain,subdomain,top_level_domain,path,path_length,num_path_segments,query_params,num_query_params,has_https,file_extension,has_fragment,has_special_chars_in_path,has_port,port_number,is_ip_address
0,https://ethnicelebs.com/eddie-anderson-comedian,benign,https,ethnicelebs.com,none,com,/eddie-anderson-comedian,24,1,0,0,1,none,0,0,0,0,0
1,https://closinglogos.com/page/Sony+Pictures+Ho...,benign,https,closinglogos.com,none,com,/page/Sony+Pictures+Home+Entertainment+Warning...,53,2,0,0,1,none,0,0,0,0,0
2,https://terezowens.com/golfer-greg-norman-tryi...,benign,https,terezowens.com,none,com,/golfer-greg-norman-trying-to-unload-55-millio...,54,1,0,0,1,none,0,0,0,0,0
3,https://americannortel.com/,benign,https,americannortel.com,none,com,/,1,0,0,0,1,none,0,0,0,0,0
4,https://welding.org/,benign,https,welding.org,none,org,/,1,0,0,0,1,none,0,0,0,0,0


In [14]:
import pandas as pd

# Load dataset
df2 = pd.read_csv("/kaggle/input/info-sec-a2/Balanced_dataset.csv")

df2 = df2.fillna("")
df2["text"] = df2["url"].astype(str) + " " + df2["scheme"].astype(str) + " " + df2["domain"].astype(str) + " " + \
              df2["subdomain"].astype(str) + " " + df2["top_level_domain"].astype(str) + " " + \
              df2["path"].astype(str) + " " + df2["file_extension"].astype(str)

# Check for unexpected non-string types
assert df2["text"].apply(lambda x: isinstance(x, str)).all(), "Error: Some values are not strings!"
# Check data

print(df2[["text", "type"]].head())


                                                text    type
0  https://ethnicelebs.com/eddie-anderson-comedia...  benign
1  https://closinglogos.com/page/Sony+Pictures+Ho...  benign
2  https://terezowens.com/golfer-greg-norman-tryi...  benign
3  https://americannortel.com/ https americannort...  benign
4  https://welding.org/ https welding.org none or...  benign


In [59]:
CLASS_MAPPING = {"benign": 0, "defacement": 1, "phishing": 2, "malware": 3, "spam": 4}
df2["label"] = df2["type"].map(CLASS_MAPPING)  # Convert text labels to numbers
data=df2[["url", "label"]]

### TOKENSIZER AND DATA LOADER

In [ ]:
def tokenize_url(url, max_length=MAX_LENGTH):
    allowed_chars = string.ascii_lowercase + string.digits + "._-/"
    url = url.lower()
    url = "".join([c if c in allowed_chars else " " for c in url])  # Replace unknown chars with space
    tokens = url.split()[:max_length]  # Tokenize by space
    char_to_idx = {char: idx + 1 for idx, char in enumerate(allowed_chars)}  # Char-level vocab
    token_ids = [char_to_idx.get(char, 0) for token in tokens for char in token]
    # Pad/Truncate
    token_ids = token_ids[:max_length] + [0] * (max_length - len(token_ids))
    return token_ids

class URLDataset(Dataset):
    def __init__(self, urls, labels, max_length=50):
        self.urls = urls
        self.labels = labels.astype(int)
        self.max_length = max_length
    def __len__(self):
        return len(self.urls)
    def __getitem__(self, idx):
        tokenized_url = torch.tensor(tokenize_url(self.urls[idx], self.max_length), dtype=torch.long)
        return {"input_ids": tokenized_url, "label": torch.tensor(self.labels[idx], dtype=torch.long)}

### TRANSFORMER MODEL CLASS

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, embed_dim, max_length=MAX_LENGTH):
        super().__init__()
        pos = torch.arange(0, max_length).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, embed_dim, 2) * (-np.log(10000.0) / embed_dim))
        pe = torch.zeros(max_length, embed_dim)
        pe[:, 0::2] = torch.sin(pos * div_term)
        pe[:, 1::2] = torch.cos(pos * div_term)
        self.pe = pe.unsqueeze(0)  # Shape: (1, max_length, embed_dim)

    def forward(self, x):
        return x + self.pe.to(x.device)

class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim=128, num_heads=4, num_layers=3, num_classes=5, max_length=50, dropout=0.3):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size + 1, embed_dim)  # Char embedding
        self.positional_encoding = PositionalEncoding(embed_dim, max_length)

        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc = nn.Linear(embed_dim, num_classes)
        self.dropout = nn.Dropout(dropout)

    def forward(self, input_ids):
        x = self.embedding(input_ids)  # Shape: (batch, seq_len, embed_dim)
        x = self.positional_encoding(x)
        x = self.transformer_encoder(x)  # Shape: (batch, seq_len, embed_dim)
        x = torch.mean(x, dim=1)  # Global average pooling
        x = self.dropout(x)
        return self.fc(x)

def train_model(model, train_loader, criterion, optimizer, epochs=3):
    model.train()
    for epoch in range(epochs):
        loop = tqdm(train_loader, desc=f"Epoch {epoch + 1}/{epochs}")
        total_loss = 0
        correct, total = 0, 0

        for batch in loop:
            input_ids = batch["input_ids"].to(DEVICE)
            labels = batch["label"].to(DEVICE)

            optimizer.zero_grad()
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            _, preds = torch.max(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            total_loss += loss.item()
            loop.set_postfix(loss=total_loss / (total + 1), acc=correct / total)

def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    total_loss = 0

    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Evaluating"):
            input_ids = batch["input_ids"].to(DEVICE)
            labels = batch["label"].to(DEVICE)

            outputs = model(input_ids)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, dim=1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            total_loss += loss.item()

    print(f"Validation Loss: {total_loss / len(test_loader):.4f}")
    print(f"Validation Accuracy: {correct / total:.4f}")



### TRAINING TESTING LOADERS

In [ ]:

train_data, test_data = train_test_split(data, test_size=0.2, stratify=data["label"], random_state=42)



train_dataset = URLDataset(train_data["text"].values, train_data["label"].values)
test_dataset = URLDataset(test_data["text"].values, test_data["label"].values)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

### MODEL INITIALISATION

In [57]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_LENGTH = 100
BATCH_SIZE = 32
EPOCHS = 8
LEARNING_RATE = 2e-4
EMBEDDING_DIM = 256
NUM_HEADS = 4
NUM_ENCODER_LAYERS = 6
DROPOUT = 0.2
VOCAB_SIZE = len(string.ascii_lowercase + string.digits + "._-/")
model = TransformerClassifier(vocab_size=VOCAB_SIZE, embed_dim=EMBEDDING_DIM, num_heads=NUM_HEADS, 
                              num_layers=NUM_ENCODER_LAYERS, num_classes=len(CLASS_MAPPING)).to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE)



/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


### MODEL TRAINING

In [58]:
train_model(model, train_loader, criterion, optimizer, epochs=EPOCHS)

Epoch 8/8: 100%|██████████| 1491/1491 [00:36<00:00, 40.86it/s, acc=0.851, loss=0.0123]


### MODEL EVALUATION

In [60]:
evaluate_model(model, test_loader)

Evaluating: 100%|██████████| 373/373 [00:02<00:00, 131.73it/s]

Validation Loss: 0.3818
Validation Accuracy: 0.8557
